In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai import *
from fastai.vision import *
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(42)

In [ ]:
PATH = Path('../input/aptos2019-blindness-detection/')

In [ ]:
df_train = pd.read_csv(PATH/'train.csv').assign(filename=lambda df:"train_images/"+df.id_code+".png")
df_test = pd.read_csv(PATH/'test.csv').assign(filename=lambda df:"test_images/"+df.id_code+".png")

In [ ]:
df_train.diagnosis.hist()

In [ ]:
transforms = get_transforms(do_flip=True,flip_vert=True,max_zoom=1.1,max_rotate=360,max_lighting=0.2,max_warp=0.2,p_lighting=0.5)

In [ ]:
data = ImageDataBunch.from_df(path=PATH,df=df_train,fn_col="filename",label_col="diagnosis",ds_tfms=transforms,size=224).normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=3,fig_size=(5,5))

In [ ]:
Path('/tmp/.cache/torch/checkpoints/').mkdir(exist_ok=True, parents=True)

In [ ]:
!cp '../input/pytorch-pretrained-image-models/resnet50.pth' '/tmp/.cache/torch/checkpoints/resnet50-19c8e357.pth'

In [ ]:
kappa = KappaScore()
kappa.weights = "quadratic"
learn = cnn_learner(data, models.resnet50,
                    metrics=[error_rate, kappa],
                    model_dir="/tmp/model/")

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(suggestion=True)

In [ ]:
lr=9e-3
learn.fit_one_cycle(10,lr)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(8,8), dpi=60)

In [ ]:
sample_df = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
sample_df.head()

In [ ]:
learn.data.add_test(ImageList.from_df(
    sample_df, PATH,
    folder='test_images',
    suffix='.png'
))

In [ ]:
preds,y = learn.get_preds(DatasetType.Test)

In [ ]:
sample_df.diagnosis = preds.argmax(1)

In [ ]:
sample_df

In [ ]:
sample_df.to_csv('submission.csv',index=False)
_ = sample_df.hist()